# Khai báo mô hình 

In [1]:
import random
import torch
import torchtext
import torch.nn as nn
import pickle

# set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [28]:
# import os
# os.chdir("Inference_seq2seq")

In [3]:
# !pip install pyvi
# !pip install https://gitlab.com/trungtv/vi_spacy/-/raw/master/vi_core_news_lg/dist/vi_core_news_lg-0.0.1.tar.gz

In [4]:
from pyvi import ViTokenizer

'''
tokenization code
'''

import spacy
spacy_vi = spacy.load('vi_core_news_lg')
stopwords = spacy_vi.Defaults.stop_words

def tokenize_vi(text):
    """
    Tokenizes Vietnamese text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_vi.tokenizer(text)]

In [5]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim,n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.dropout = dropout
        self.n_layers = n_layers

        # self.embedding = nn.Embedding(input_dim, emb_dim)
        # further you can now make use of your pretrained embeddings inside your model by passing them in
        # as a parameter or loading the handy pickle file mentioned above - a much better design pattern
        # indeed than relying on TEXT.build_vocab() in order to define the embedding layer of your model
        self.embedding = nn.Embedding.from_pretrained(SRC_saved.vocab.vectors, freeze=False)
        # freeze=True <-> self.embedding.requires_grad = False, not learn in training
        # SRC.vocab.vectors, freeze=False

        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, n_layers, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.lstm(embedded)
        
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, dec_hid_dim, n_layers, dropout):
        super().__init__()

        self.emb_dim = emb_dim
        self.output_dim = output_dim
        self.dec_hid_dim = dec_hid_dim
        self.n_layers = n_layers
        self.dropout = dropout

        # self.embedding = nn.Embedding(output_dim, emb_dim)
        self.embedding = nn.Embedding.from_pretrained(TRG_saved.vocab.vectors, freeze=False)
        # self.embedding.requires_grad = False
        self.lstm = nn.LSTM(emb_dim, dec_hid_dim, n_layers, dropout=dropout)
        self.fc_out = nn.Linear(dec_hid_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]    
        output, (hidden, cell) = self.lstm(embedded, (hidden, cell))
        
        # predicted shape is [batch_size, output_dim]
        prediction = self.fc_out(hidden.squeeze(0))
        
        return prediction, hidden, cell

class Seq2Seq(nn.Module):
    ''' 
    Args:
        encoder: A Encoder class instance.
        decoder: A Decoder class instance.
    '''
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        # to store the outputs of the decoder
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # first input to the decoder is the <sos> tokens
        input = trg[0, :]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            use_teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if use_teacher_force else top1)

        # outputs is of shape [sequence_len, batch_size, output_dim]
        return outputs

In [6]:
'''
load fields saved during preprocessing
'''
with open("TRG.Field","rb") as f:
     TRG_saved = pickle.load(f)

with open("SRC.Field","rb") as f:
     SRC_saved = pickle.load(f)

'''
hyperparameters (ensure the following hyperparameters match with those used during training of the best model)
'''
INPUT_DIM = len(SRC_saved.vocab)
OUTPUT_DIM = len(TRG_saved.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
N_LAYERS = 1

'''
instantiate the model
'''
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, N_LAYERS, DEC_DROPOUT)
model_best = Seq2Seq(enc, dec, device).to(device)

C:\Users\PC\miniconda3\envs\myenv\lib\site-packages\torch\nn\modules\rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [11]:
'''
load the checkpoint corresponding to the best epoch (usually epoch with highest validation BLEU score)
'''
# n_ckpt = 200
n_ckpt = 237
model_best.load_state_dict(torch.load(r'C:\Users\PC\Desktop\Vietnamese_Paraphrased_Generator\Generator\ckpt_seq2seq\seq2seq_237.pt')['state_dict'])
model_best = model_best.to(device)

In [17]:
test_data = torchtext.legacy.data.TabularDataset(
    path="test_paraphrase.csv", 
    format='csv', skip_header=True, fields=[('src', SRC_saved)]) 

In [25]:
data = ["Facebook là một trang mạng xã hội tuyệt vời"]
a = torchtext.legacy.data.Example.fromlist(data, fields=[('src', SRC_saved)])

In [27]:
a.get

<method-wrapper '__getattribute__' of Example object at 0x000002440DEB70B8>

In [19]:
test_iter = torchtext.legacy.data.Iterator(test_data, batch_size=16, device=device, sort=False, sort_key=None, shuffle=False, sort_within_batch=False)

In [20]:
test_iter

In [21]:
# convert index to text string
def convert_itos(convert_vocab, token_ids):
    list_string = []
    for i in token_ids:
        if i == convert_vocab.vocab.stoi['<eos>']:
            break
        else:
            token = convert_vocab.vocab.itos[i]
            list_string.append(token)
    return list_string

In [23]:
TRG_PAD_IDX = TRG_saved.vocab.stoi[TRG_saved.pad_token]
print(TRG_PAD_IDX)

1


In [ ]:
'''
generate paraphrases for all the sentences in test data
'''
def generate_paraphrases(model, SRC_saved, TRG_saved, eval_iter, trg_vocab, attention = True, max_trg_len = 64):
    model.eval()
    TRG_PAD_IDX = TRG_saved.vocab.stoi[TRG_saved.pad_token]
    all_translation_word_ids = []
    for batch in test_iter:
        src = batch.src
        batch_size = src.shape[1]


        trg_placeholder = torch.Tensor(max_trg_len, batch_size)
        trg_placeholder.fill_(TRG_PAD_IDX)
        trg_placeholder = trg_placeholder.long().to(device)
        if attention == True:
          output,_ = model(src, trg_placeholder, 0) #turn off teacher forcing
        else:
          output = model(src, trg_placeholder, 0) #turn off teacher forcing
        
        output_translate = output[1:]

        # Choose top 1 word from decoder's output, we get the probability and index of the word
        prob, token_id = output_translate.data.topk(1)
        translation_token_id = token_id.squeeze(2).cpu()

        # store gold target sentences to a list 
        all_translation_word_ids.append(translation_token_id)

    all_translation_text = []
    for i in range(len(all_translation_word_ids)):
      cur_translation_batch = all_translation_word_ids[i]
      for j in range(cur_translation_batch.shape[1]):
        trans_convered_strings = convert_itos(trg_vocab, cur_translation_batch[:,j])
        all_translation_text.append(' '.join(trans_convered_strings)) # convert list of words to text
    
    return all_translation_text

In [ ]:
# Output đầu ra là các câu đồng nghĩa
# output all the sentences in the test set    
test_predictions = generate_paraphrases(model_best, test_iter, TRG_saved, attention = False, max_trg_len = 64)

In [ ]:
import pandas as pd
test_csv = pd.read_csv("test_paraphrase.csv")["src"].tolist()
test_csv

['Để bán được hàng thàng công , trước tiên bạn phải yêu quý công việc và hết lòng vì nó . ',
 'Điều này cho phép bạn lựa chọn những công việc cần phải hoàn thành trong ngày hôm đó và loại bỏ các công việc có thể hoàn thành vào các ngày khác .',
 'Vì vậy , bạn hãy học cách nói “ không ” với những yêu cầu có mức ưu tiên thấp , và bạn sẽ có nhiều thời gian để làm những việc quan trọng hơn .',
 'Để có được những thành tựu như ngày hôm nay , ngoài những giải pháp chung đã được nhiều tỉnh thành trên cả nước áp dụng , Đà Nẵng còn có những “ chiêu ” chưa hề có tiền lệ , góp phần phát triển kinh tế một cách mạnh mẽ và tạo ra sự ổn định trong đời sống xã hội .',
 'Không dừng lại ở bạn bè , thầy cô , mà ngay cả người thân trong gia đình cũng bị không ít bạn trẻ dùng những từ ngữ “ vô học ” công kích trên mạng . ',
 'Chúng ta có nhiều giống cây , con bản địa quý , mang hương vị Việt Nam , nhưng đã bị lãng quên nhiều năm , và nhiều giống đã vĩnh viễn biến mất . ',
 'Còn nhà mạng , vốn cũng là một d

In [ ]:
for (orig_sent, pred_sent) in zip(test_csv, test_predictions):
  print("Orig: ", orig_sent)
  print("Pred: ", pred_sent)

Streaming output truncated to the last 5000 lines.
Orig:  Vào năm 1975, một thuật toán cố định khác được phát triển bởi Yasuo Sugi, dựa trên phương pháp cải tiến.
Pred:  vào năm 1942 , một sự một một phục của 22 trong , - 0 và chính_phủ - a , và ( đi - của nít trong ) của a .
Orig:  Black Bull cun (` 1. 9% ABV '' ) đã được tổ chức để tưởng niệm lần đầu tiên của Theokson quán rượu nơi cốc đầu tiên của Thekston's đầu tiên đã được pha loãng và bán.
Pred:  black bulll cun mà thương_mại nhật_bản . bán % abv hay dễ năm 2013 , ông ngửa ra đến để công_nhân công_nghệ , , mà ) xe được được biệt_động hết phía phía % loại % usd , để các usd một kiến_thức cho một nửa bằng bằng , một các khác .
Orig:  Nguyên văn có chữ đen trên nền trắng.
Pred:  lại có các đủ ra người như viết chuẩn_bị tỏ " quốc_hội chữ - , và từ ra khi để liên_quan , như mà , rằng đang trên viết của nền và ” là người trên được trên mỗi được gặp để để .
Orig:  Ông đã chơi Lollalalalalalalooza ở Chicago vào năm 2007, và Fuck Ya Festi

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
test_predictions

['game phải được từ bởi thuộc và liên_quan giữa thứ của và vào vào việc giới_hạn 1 mỹ đặc_biệt chính người tên nghiên_cứu sử_dụng nội_dung - ri việt_nam phủ_sóng tính_toán tốt đi họ họ xã_hội gái sử_dụng sử_dụng viral marketing ) , không không cộng_đồng cộng_đồng về đông_đảo đông_đảo đông_đảo đông_đảo không_thể .',
 'game trợ_giá phải như hoa_kỳ bộ không chỉ_số quan_trọng như_vậy vn việc đưa , bằng họ cung_cấp ra dịch_vụ liên_quan mà vì 156 / 189 tốt thế_giới chỉ_số apple đến thương_hiệu và khẳng_định trăm và trong môi_trường trong ( đây đây',
 'hiện_nay , , đảm_bảo hiệu_quả làn_sóng dịch_chuyển xuất_hiện như bài_toán nhiều quan_trọng , các nhiều họ và có mối béo cao và tiếp_cận điện khác ngành công_nghiệp , chia hiện_nay , đôi trên chính_sách , , xử_lý xử_lý , cái cái môi_trường môi_trường môi_trường .',
 'các trung_quốc « và « sản_xuất - sản_xuất , - ma - vào , sau , ma - hai nữ qua , đưa - ri - thừa_nhận , câu , để các quốc_tế , và có_thể đưa đưa kể_cả và giữa giữa là .',
 'hãy ở nh

In [ ]:
type(test_iter)

torchtext.legacy.data.iterator.Iterator

In [ ]:
for batch in test_iter:
  src = batch.src
  # trg = batch.trg
  print(type(batch))
  batch
  break

print(src)
print(src.shape)

<class 'torchtext.legacy.data.batch.Batch'>
tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     2,     2,
             2,     2,     2,     2,     2,     2],
        [   21,   201,   838,    21,    42,   412,   113,   314,  1111,    81,
            27,   548,  4261,    17,   672,   201],
        [  254,    55,     5,    36,  1315,    36,    84,    49,   212,   212,
             9,    36,    66,    46,     5,   477],
        [   18,    35,    97,    18,    90,    69,   461,   123,   108,     5,
          8643,   463,    15,    66,    69,   132],
        [  205,   669,   577,    25,    12,   561,     5,    15,   392,  2811,
             0,    54,    99,   312,  1499,    20],
        [    0,    97,   188,     0,  1294,   353,   799,     0,    15,    15,
             5,    62,    95,    20,    79,    97],
        [ 1767,  1147,   166,    79,     5,     5,    75,   277,    17,   211,
            46,   356,   106,   731,   119,   577],
        [    5,    25,   212,    57,   

In [ ]:
torchtext.data.Example(fields=[('src', SRC_saved)])